In [54]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 数据预处理类
# 修复数据预处理类中的日期处理问题
class WeatherDataProcessor:
    def __init__(self):
        self.scalers = {}
        self.label_encoders = {}
        
    def load_and_preprocess_data(self, file_path):
        """加载和预处理气象数据"""
        # 读取数据
        df = pd.read_csv(file_path)
        
        # 打印列名以便调试
        print("数据集列名:", df.columns.tolist())
        print("前5行数据:")
        print(df.head())
        
        # 处理日期列 - 修复错误
        try:
            # 尝试转换日期列
            df['Formatted Date1'] = pd.to_datetime(df['Formatted Date'], errors='coerce')
            
            # 检查转换是否成功
            if df['Formatted Date1'].isna().all():
                print("日期转换失败，尝试其他格式...")
                # 尝试不同的日期格式
                df['Formatted Date1'] = pd.to_datetime(df['Formatted Date'], 
                                                      format='%Y-%m-%d %H:%M:%S.%f %z', 
                                                      errors='coerce')
                
            # 如果仍然失败，尝试更简单的格式
            if df['Formatted Date1'].isna().all():
                print("尝试简化日期格式...")
                # 移除时区信息再转换
                df['date_clean'] = df['Formatted Date'].str.replace(r' \+\d{4}$', '', regex=True)
                df['Formatted Date1'] = pd.to_datetime(df['date_clean'], errors='coerce')
                
            # 检查是否还有问题
            na_count = df['Formatted Date1'].isna().sum()
            if na_count > 0:
                print(f"警告: {na_count} 个日期转换失败，将使用默认值")
                # 为失败的日期设置默认值
                df['Formatted Date1'] = df['Formatted Date1'].fillna(pd.Timestamp('2020-01-01'))
                
            # 提取日期特征
            df['Year'] = df['Formatted Date1'].dt.year
            df['Month'] = df['Formatted Date1'].dt.month
            df['Day'] = df['Formatted Date1'].dt.day
            df['Hour'] = df['Formatted Date1'].dt.hour
            
        except Exception as e:
            print(f"日期处理出错: {e}")
            # 如果日期处理完全失败，创建虚拟的时间特征
            print("使用虚拟时间特征...")
            df['Year'] = 2020
            df['Month'] = 1
            df['Day'] = 1
            df['Hour'] = 0
        
        # 处理分类变量
        categorical_cols = ['Summary', 'Precip Type']
        for col in categorical_cols:
            if col in df.columns:
                print(f"处理分类变量: {col}")
                le = LabelEncoder()
                # 处理缺失值
                df[col] = df[col].fillna('unknown')
                df[col] = le.fit_transform(df[col])
                self.label_encoders[col] = le
        
        # 选择数值特征
        numeric_cols = ['Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 
                       'Wind Speed (km/h)', 'Wind Bearing (degrees)', 'Visibility (km)',
                       'Loud Cover', 'Pressure (millibars)', 'Year', 'Month', 'Day', 'Hour']
        
        # 检查哪些列实际存在
        existing_numeric_cols = [col for col in numeric_cols if col in df.columns]
        existing_categorical_cols = [col for col in categorical_cols if col in df.columns]
        
        print(f"存在的数值列: {existing_numeric_cols}")
        print(f"存在的分类列: {existing_categorical_cols}")
        
        # 添加分类特征
        feature_cols = existing_numeric_cols + existing_categorical_cols
        
        # 处理缺失值
        for col in feature_cols:
            if col in df.columns:
                if df[col].dtype in ['int64', 'float64']:
                    median_val = df[col].median()
                    if pd.isna(median_val):
                        median_val = 0
                    df[col] = df[col].fillna(median_val)
                else:
                    mode_val = df[col].mode()
                    fill_val = mode_val[0] if not mode_val.empty else 0
                    df[col] = df[col].fillna(fill_val)
        
        # 标准化数值特征
        for col in existing_numeric_cols:
            if col in df.columns:
                try:
                    scaler = StandardScaler()
                    values = df[col].values.reshape(-1, 1)
                    df[col] = scaler.fit_transform(values).flatten()
                    self.scalers[col] = scaler
                except Exception as e:
                    print(f"标准化列 {col} 时出错: {e}")
                    continue
        
        # 确保返回的数据没有缺失值
        final_data = df[feature_cols].fillna(0)
        print(f"最终特征列: {feature_cols}")
        print(f"最终数据形状: {final_data.shape}")
        print(f"缺失值检查: {final_data.isna().sum().sum()}")
        
        return final_data.values, df
    
    def create_sequences(self, data, seq_length, pred_length, target_indices):
        """创建时序数据序列"""
        X, y = [], []
        for i in range(len(data) - seq_length - pred_length + 1):
            X.append(data[i:(i + seq_length)])
            y.append(data[(i + seq_length):(i + seq_length + pred_length), target_indices])
        return np.array(X), np.array(y)

# 数据集类
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

# 标准Transformer模型
class WeatherTransformer(nn.Module):
    def __init__(self, input_dim, d_model=256, nhead=8, num_layers=6, 
                 seq_length=168, pred_length=24, target_dim=1, dropout=0.1):
        super(WeatherTransformer, self).__init__()
        
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.target_dim = target_dim
        
        # 输入投影层
        self.input_projection = nn.Linear(input_dim, d_model)
        
        # 位置编码
        self.pos_encoding = PositionalEncoding(d_model, max_len=seq_length + pred_length)
        
        # 层归一化
        self.layer_norm = nn.LayerNorm(d_model)
        
        # Transformer编码器
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True,
            activation='gelu'  # 使用GELU激活函数
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 输出投影层 - 添加更多正则化
        self.output_projection = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, d_model // 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 4, target_dim * pred_length)
        )
        
        # 初始化权重
        self._init_weights()
        
    def _init_weights(self):
        """初始化模型权重"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                # 使用更保守的Xavier正态分布初始化
                nn.init.xavier_normal_(module.weight, gain=0.02)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.LayerNorm):
                nn.init.constant_(module.bias, 0)
                nn.init.constant_(module.weight, 1.0)
            elif isinstance(module, nn.MultiheadAttention):
                # 特别处理注意力层
                if hasattr(module, 'in_proj_weight') and module.in_proj_weight is not None:
                    nn.init.xavier_normal_(module.in_proj_weight, gain=0.02)
                if hasattr(module, 'out_proj') and hasattr(module.out_proj, 'weight'):
                    nn.init.xavier_normal_(module.out_proj.weight, gain=0.02)
        
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_dim)
        batch_size = x.size(0)
        
        # 输入投影和归一化
        x = self.input_projection(x)  # (batch_size, seq_length, d_model)
        x = self.layer_norm(x)
        
        # 添加位置编码
        x = x.transpose(0, 1)  # (seq_length, batch_size, d_model)
        x = self.pos_encoding(x)
        x = x.transpose(0, 1)  # (batch_size, seq_length, d_model)
        
        # Transformer编码
        transformer_output = self.transformer_encoder(x)  # (batch_size, seq_length, d_model)
        
        # 全局平均池化而不是只取最后一个时间步
        pooled_output = transformer_output.mean(dim=1)  # (batch_size, d_model)
        
        # 输出投影
        output = self.output_projection(pooled_output)  # (batch_size, target_dim * pred_length)
        output = output.view(batch_size, self.pred_length, self.target_dim)
        
        return output

# 改进的Transformer模型 - 引入局部注意力
class LocalAttentionTransformer(nn.Module):
    def __init__(self, input_dim, d_model=256, nhead=8, num_layers=6, 
                 seq_length=168, pred_length=24, target_dim=1, dropout=0.1, 
                 local_window=48):
        super(LocalAttentionTransformer, self).__init__()
        
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.target_dim = target_dim
        self.local_window = local_window
        
        # 输入投影层
        self.input_projection = nn.Linear(input_dim, d_model)
        
        # 位置编码
        self.pos_encoding = PositionalEncoding(d_model, max_len=seq_length + pred_length)
        
        # 自定义局部注意力层
        self.local_attention_layers = nn.ModuleList([
            LocalAttentionLayer(d_model, nhead, local_window, dropout)
            for _ in range(num_layers)
        ])
        
        # 层归一化
        self.layer_norms = nn.ModuleList([nn.LayerNorm(d_model) for _ in range(num_layers)])
        
        # 前馈网络
        self.feed_forwards = nn.ModuleList([
            nn.Sequential(
                nn.Linear(d_model, d_model * 4),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(d_model * 4, d_model),
                nn.Dropout(dropout)
            ) for _ in range(num_layers)
        ])
        
        # 输出投影层
        self.output_projection = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, target_dim * pred_length)
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # 输入投影
        x = self.input_projection(x)
        
        # 添加位置编码
        x = x.transpose(0, 1)
        x = self.pos_encoding(x)
        x = x.transpose(0, 1)
        
        # 局部注意力层
        for i, (local_attn, layer_norm, ff) in enumerate(zip(
            self.local_attention_layers, self.layer_norms, self.feed_forwards
        )):
            # 局部注意力
            attn_output = local_attn(x)
            x = layer_norm(x + attn_output)
            
            # 前馈网络
            ff_output = ff(x)
            x = layer_norm(x + ff_output)
        
        # 取最后一个时间步的输出
        last_output = x[:, -1, :]
        
        # 输出投影
        output = self.output_projection(last_output)
        output = output.view(batch_size, self.pred_length, self.target_dim)
        
        return output

# 局部注意力层
class LocalAttentionLayer(nn.Module):
    def __init__(self, d_model, nhead, local_window, dropout=0.1):
        super(LocalAttentionLayer, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.local_window = local_window
        self.head_dim = d_model // nhead
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        Q = self.q_linear(x).view(batch_size, seq_len, self.nhead, self.head_dim).transpose(1, 2)
        K = self.k_linear(x).view(batch_size, seq_len, self.nhead, self.head_dim).transpose(1, 2)
        V = self.v_linear(x).view(batch_size, seq_len, self.nhead, self.head_dim).transpose(1, 2)
        
        # 创建局部注意力掩码
        mask = self.create_local_mask(seq_len, self.local_window, x.device)
        
        # 计算注意力分数
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        scores = scores.masked_fill(mask == 0, -1e9)
        
        attention_weights = torch.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # 应用注意力
        attended = torch.matmul(attention_weights, V)
        attended = attended.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        
        output = self.out_linear(attended)
        return output
    
    def create_local_mask(self, seq_len, window_size, device):
        mask = torch.zeros(seq_len, seq_len, device=device)
        for i in range(seq_len):
            start = max(0, i - window_size // 2)
            end = min(seq_len, i + window_size // 2 + 1)
            mask[i, start:end] = 1
        return mask.unsqueeze(0).unsqueeze(0)  # 添加batch和head维度

In [55]:

# CNN-Transformer混合模型
class CNNTransformer(nn.Module):
    def __init__(self, input_dim, d_model=256, nhead=8, num_layers=4, 
                 seq_length=168, pred_length=24, target_dim=1, dropout=0.1):
        super(CNNTransformer, self).__init__()
        
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.target_dim = target_dim
        
        # CNN特征提取层
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, d_model, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(d_model),
            nn.Dropout(dropout)
        )
        
        # 位置编码
        self.pos_encoding = PositionalEncoding(d_model, max_len=seq_length)
        
        # Transformer编码器
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 输出投影层
        self.output_projection = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, target_dim * pred_length)
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # CNN特征提取
        x_cnn = x.transpose(1, 2)  # (batch_size, input_dim, seq_length)
        cnn_features = self.cnn_layers(x_cnn)  # (batch_size, d_model, seq_length)
        cnn_features = cnn_features.transpose(1, 2)  # (batch_size, seq_length, d_model)
        
        # 添加位置编码
        x_pos = cnn_features.transpose(0, 1)  # (seq_length, batch_size, d_model)
        x_pos = self.pos_encoding(x_pos)
        x_pos = x_pos.transpose(0, 1)  # (batch_size, seq_length, d_model)
        
        # Transformer编码
        transformer_output = self.transformer_encoder(x_pos)
        
        # 取最后一个时间步的输出
        last_output = transformer_output[:, -1, :]
        
        # 输出投影
        output = self.output_projection(last_output)
        output = output.view(batch_size, self.pred_length, self.target_dim)
        
        return output

# LSTM对比模型
class WeatherLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_layers=3, 
                 pred_length=24, target_dim=1, dropout=0.1):
        super(WeatherLSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.pred_length = pred_length
        self.target_dim = target_dim
        
        # LSTM层
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        
        # 输出层
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, target_dim * pred_length)
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # LSTM前向传播
        lstm_out, (hidden, cell) = self.lstm(x)
        
        # 取最后一个时间步的输出
        last_output = lstm_out[:, -1, :]
        
        # 输出投影
        output = self.output_layer(last_output)
        output = output.view(batch_size, self.pred_length, self.target_dim)
        
        return output

def create_safe_scheduler(optimizer):
    """安全地创建学习率调度器，兼容不同PyTorch版本"""
    try:
        # 尝试使用verbose参数
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.7, patience=5, verbose=True, min_lr=1e-7
        )
        return scheduler
    except TypeError:
        # 如果不支持verbose参数
        print("注意：使用兼容模式的学习率调度器")
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.7, patience=5, min_lr=1e-7
        )
        return scheduler
    
# 训练器类
class ModelTrainer:
    def __init__(self, model, device, learning_rate=0.00005, weight_decay=1e-4):
        self.model = model.to(device)
        self.device = device
        
        # 创建优化器
        self.optimizer = optim.AdamW(
            model.parameters(), 
            lr=learning_rate, 
            weight_decay=weight_decay, 
            betas=(0.9, 0.999)
        )
        
        # 使用安全的调度器创建函数
        self.scheduler = create_safe_scheduler(self.optimizer)
        
        # 损失函数
        self.criterion = nn.MSELoss(reduction='mean')
        self.train_losses = []
        self.val_losses = []
        
        # 初始化权重
        self._init_weights()
        
    def _init_weights(self):
        """改进的模型权重初始化"""
        for name, param in self.model.named_parameters():
            if 'weight' in name:
                if len(param.shape) >= 2:
                    # 使用更保守的Xavier初始化
                    nn.init.xavier_uniform_(param, gain=0.1)
                else:
                    # 1D权重使用更小的正态分布初始化
                    nn.init.normal_(param, 0, 0.01)
            elif 'bias' in name:
                nn.init.constant_(param, 0)
        
        # 对于Transformer特定层的特殊初始化
        for module in self.model.modules():
            if isinstance(module, nn.MultiheadAttention):
                # 注意力层的权重使用更小的初始化
                if hasattr(module, 'in_proj_weight') and module.in_proj_weight is not None:
                    nn.init.xavier_uniform_(module.in_proj_weight, gain=0.1)
                if hasattr(module, 'out_proj') and module.out_proj.weight is not None:
                    nn.init.xavier_uniform_(module.out_proj.weight, gain=0.1)
        
    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        batch_count = 0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
            
            # 检查输入数据是否有异常值
            if torch.isnan(batch_x).any() or torch.isinf(batch_x).any():
                print("警告：输入数据包含NaN或Inf值，跳过此批次")
                continue
                
            self.optimizer.zero_grad()
            
            try:
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                
                # 检查损失是否异常
                if torch.isnan(loss) or torch.isinf(loss):
                    print("警告：损失为NaN或Inf，跳过此批次")
                    continue
                
                loss.backward()
                
                # 添加更强的梯度裁剪防止梯度爆炸
                grad_norm = torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)
                
                # 检查梯度是否过大
                if grad_norm > 10.0:
                    print(f"警告：梯度范数过大 {grad_norm:.2f}，跳过此次更新")
                    continue
                
                self.optimizer.step()
                
                total_loss += loss.item()
                batch_count += 1
                
            except RuntimeError as e:
                print(f"训练过程中出现错误: {e}")
                continue
        
        if batch_count == 0:
            return float('inf')
        
        avg_loss = total_loss / batch_count
        self.train_losses.append(avg_loss)
        return avg_loss
    
    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0
        
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                total_loss += loss.item()
        
        avg_loss = total_loss / len(val_loader)
        self.val_losses.append(avg_loss)
        return avg_loss
    
    def train(self, train_loader, val_loader, epochs=30, patience=8):
        best_val_loss = float('inf')
        patience_counter = 0
        
        print(f"开始训练，初始学习率: {self.optimizer.param_groups[0]['lr']:.6f}")
        
        for epoch in range(epochs):
            train_loss = self.train_epoch(train_loader)
            val_loss = self.validate(val_loader)
            
            # 学习率调度
            old_lr = self.optimizer.param_groups[0]['lr']
            self.scheduler.step(val_loss)
            new_lr = self.optimizer.param_groups[0]['lr']
            
            if new_lr != old_lr:
                print(f"学习率从 {old_lr:.6f} 降低到 {new_lr:.6f}")
            
            print(f'Epoch {epoch+1}/{epochs}: Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, LR: {new_lr:.6f}')
            
            # 检查是否出现异常情况
            if torch.isnan(torch.tensor(train_loss)) or torch.isnan(torch.tensor(val_loss)):
                print("检测到NaN损失，停止训练")
                break
                
            if train_loss > 1000 or val_loss > 1000:
                print("损失过大，可能出现梯度爆炸，停止训练")
                break
            
            # 检查训练是否稳定
            if epoch > 5 and len(self.train_losses) >= 5:
                recent_losses = self.train_losses[-5:]
                if all(recent_losses[i] > recent_losses[i-1] for i in range(1, len(recent_losses))):
                    print("连续5个epoch损失都在增加，可能学习率过高，降低学习率")
                    for param_group in self.optimizer.param_groups:
                        param_group['lr'] *= 0.5
                    print(f"学习率降低到: {self.optimizer.param_groups[0]['lr']:.6f}")
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # 保存最佳模型
                torch.save(self.model.state_dict(), 'best_model.pth')
                print(f"保存最佳模型，验证损失: {val_loss:.6f}")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break
        
        # 加载最佳模型
        try:
            self.model.load_state_dict(torch.load('best_model.pth', map_location=self.device))
            print("成功加载最佳模型")
        except:
            print("无法加载最佳模型，使用当前模型")
        return self.model

# 模型评估类
class ModelEvaluator:
    def __init__(self, model, device, scaler=None):
        self.model = model
        self.device = device
        self.scaler = scaler
        
    def predict(self, test_loader):
        self.model.eval()
        predictions = []
        actuals = []
        
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                outputs = self.model(batch_x)
                
                predictions.append(outputs.cpu().numpy())
                actuals.append(batch_y.cpu().numpy())
        
        predictions = np.concatenate(predictions, axis=0)
        actuals = np.concatenate(actuals, axis=0)
        
        return predictions, actuals
    
    def evaluate_metrics(self, predictions, actuals):
        # 计算各种评估指标
        metrics = {}
        
        # 对每个预测步长计算指标
        for step in range(predictions.shape[1]):
            pred_step = predictions[:, step, :]
            actual_step = actuals[:, step, :]
            
            mse = mean_squared_error(actual_step, pred_step)
            mae = mean_absolute_error(actual_step, pred_step)
            rmse = np.sqrt(mse)
            
            # 计算R²分数
            try:
                r2 = r2_score(actual_step, pred_step)
            except:
                r2 = 0.0
            
            metrics[f'step_{step+1}'] = {
                'MSE': mse,
                'MAE': mae,
                'RMSE': rmse,
                'R2': r2
            }
        
        # 计算总体指标
        pred_flat = predictions.reshape(-1, predictions.shape[-1])
        actual_flat = actuals.reshape(-1, actuals.shape[-1])
        
        overall_mse = mean_squared_error(actual_flat, pred_flat)
        overall_mae = mean_absolute_error(actual_flat, pred_flat)
        overall_rmse = np.sqrt(overall_mse)
        
        try:
            overall_r2 = r2_score(actual_flat, pred_flat)
        except:
            overall_r2 = 0.0
        
        metrics['overall'] = {
            'MSE': overall_mse,
            'MAE': overall_mae,
            'RMSE': overall_rmse,
            'R2': overall_r2
        }
        
        return metrics
    
    def plot_predictions(self, predictions, actuals, feature_names, save_path=None):
        """绘制预测结果"""
        n_features = predictions.shape[-1]
        n_samples = min(100, predictions.shape[0])  # 只显示前100个样本
        
        fig, axes = plt.subplots(n_features, 1, figsize=(15, 4*n_features))
        if n_features == 1:
            axes = [axes]
        
        for i in range(n_features):
            for j in range(n_samples):
                if j == 0:
                    axes[i].plot(predictions[j, :, i], 'r-', alpha=0.6, label='Predicted')
                    axes[i].plot(actuals[j, :, i], 'b-', alpha=0.6, label='Actual')
                else:
                    axes[i].plot(predictions[j, :, i], 'r-', alpha=0.1)
                    axes[i].plot(actuals[j, :, i], 'b-', alpha=0.1)
            
            axes[i].set_title(f'{feature_names[i]} Prediction vs Actual')
            axes[i].set_xlabel('Time Steps')
            axes[i].set_ylabel('Value')
            axes[i].legend()
            axes[i].grid(True)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()

# 超参数调优类
class HyperparameterTuner:
    def __init__(self, model_class, train_loader, val_loader, device):
        self.model_class = model_class
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.results = []
        
    def tune_hyperparameters(self, param_grid, input_dim, target_dim=1):
        """网格搜索超参数调优"""
        best_score = float('inf')
        best_params = None
        
        # 生成参数组合
        param_combinations = self._generate_param_combinations(param_grid)
        
        for i, params in enumerate(param_combinations):
            print(f"\n调优进度: {i+1}/{len(param_combinations)}")
            print(f"当前参数: {params}")
            
            try:
                # 分离模型参数和训练参数
                model_params = {}
                training_params = {}
                
                for key, value in params.items():
                    if key == 'learning_rate':
                        training_params[key] = value
                    else:
                        model_params[key] = value
                
                # 创建模型 - 只传递模型相关的参数
                model = self.model_class(
                    input_dim=input_dim, 
                    target_dim=target_dim, 
                    **model_params
                )
                
                # 训练模型 - 传递学习率参数
                learning_rate = training_params.get('learning_rate', 0.001)
                trainer = ModelTrainer(model, self.device, learning_rate=learning_rate)
                trainer.train(self.train_loader, self.val_loader, epochs=20, patience=5)
                
                # 评估模型
                val_loss = trainer.validate(self.val_loader)
                
                # 记录结果
                result = {'params': params, 'val_loss': val_loss}
                self.results.append(result)
                
                print(f"验证损失: {val_loss:.6f}")
                
                # 更新最佳参数
                if val_loss < best_score:
                    best_score = val_loss
                    best_params = params
                    print(f"发现更好的参数组合! 损失: {val_loss:.6f}")
                
            except Exception as e:
                print(f"参数组合失败: {e}")
                continue
        
        return best_params, best_score, self.results
    
    def _generate_param_combinations(self, param_grid):
        """生成参数组合"""
        keys = list(param_grid.keys())
        values = list(param_grid.values())
        
        combinations = []
        def generate(index, current_params):
            if index == len(keys):
                combinations.append(current_params.copy())
                return
            
            key = keys[index]
            for value in values[index]:
                current_params[key] = value
                generate(index + 1, current_params)
                del current_params[key]
        
        generate(0, {})
        return combinations

In [56]:
print(torch.cuda.is_available())

True


In [57]:
# 修复快速超参数调优函数
def quick_hyperparameter_tuning(experiment, target_variable='Temperature'):
    """快速超参数调优 - 使用更小的搜索空间"""
    print(f"\n开始 {target_variable} 的快速超参数调优...")
    
    # 获取数据加载器
    train_loader = experiment.dataloaders[target_variable]['train']
    val_loader = experiment.dataloaders[target_variable]['val']
    
    # 定义更小的搜索空间
    param_combinations = [
        {'d_model': 128, 'nhead': 4, 'num_layers': 3, 'learning_rate': 0.001, 'dropout': 0.1},
        {'d_model': 256, 'nhead': 8, 'num_layers': 6, 'learning_rate': 0.001, 'dropout': 0.1},
        {'d_model': 256, 'nhead': 8, 'num_layers': 4, 'learning_rate': 0.01, 'dropout': 0.2},
        {'d_model': 128, 'nhead': 4, 'num_layers': 6, 'learning_rate': 0.001, 'dropout': 0.2},
    ]
    
    best_score = float('inf')
    best_params = None
    results = []
    
    for i, params in enumerate(param_combinations):
        print(f"\n调优进度: {i+1}/{len(param_combinations)}")
        print(f"当前参数: {params}")
        
        try:
            # 分离模型参数和训练参数
            model_params = {k: v for k, v in params.items() if k != 'learning_rate'}
            learning_rate = params['learning_rate']
            
            # 创建模型
            model = WeatherTransformer(
                input_dim=experiment.input_dim,
                target_dim=1,
                seq_length=experiment.seq_length,
                pred_length=experiment.pred_length,
                **model_params
            )
            
            # 训练模型
            trainer = ModelTrainer(model, experiment.device, learning_rate=learning_rate)
            trainer.train(train_loader, val_loader, epochs=15, patience=5)
            
            # 评估模型
            val_loss = trainer.validate(val_loader)
            
            # 记录结果
            result = {'params': params, 'val_loss': val_loss}
            results.append(result)
            
            print(f"验证损失: {val_loss:.6f}")
            
            # 更新最佳参数
            if val_loss < best_score:
                best_score = val_loss
                best_params = params
                print(f"发现更好的参数组合! 损失: {val_loss:.6f}")
                
        except Exception as e:
            print(f"参数组合失败: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    return best_params, best_score, results

# 修复WeatherForecastingExperiment类，移除重复的方法定义
class WeatherForecastingExperiment:
    def __init__(self, data_path, seq_length=168, pred_length=24, test_size=0.2, val_size=0.1):
        self.data_path = data_path
        self.seq_length = seq_length  # 7天的小时数据作为输入
        self.pred_length = pred_length  # 预测未来24小时
        self.test_size = test_size
        self.val_size = val_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        print(f"使用设备: {self.device}")
        
        # 初始化数据处理器
        self.data_processor = WeatherDataProcessor()
        
        # 目标变量索引
        self.target_variables = {
            'Temperature': 0,  # 温度
            'Humidity': 2,     # 湿度
            'Wind Speed': 3,   # 风速
            'Pressure': 7      # 气压
        }
        
    def load_and_prepare_data(self):
        """加载和准备数据"""
        print("正在加载和预处理数据...")
        
        # 加载数据
        data, df = self.data_processor.load_and_preprocess_data(self.data_path)
        self.feature_names = list(df.columns)
        
        print(f"数据形状: {data.shape}")
        print(f"特征数量: {len(self.feature_names)}")
        
        # 保存原始数据用于分析
        self.raw_data = data
        self.input_dim = data.shape[1]
        
        # 为每个目标变量准备数据
        self.datasets = {}
        self.dataloaders = {}
        
        for var_name, var_idx in self.target_variables.items():
            print(f"\n准备 {var_name} 的数据...")
            
            # 创建序列数据
            X, y = self.data_processor.create_sequences(
                data, self.seq_length, self.pred_length, [var_idx]
            )
            
            print(f"序列数据形状 - X: {X.shape}, y: {y.shape}")
            
            # 分割数据集
            train_size = int(len(X) * (1 - self.test_size - self.val_size))
            val_size = int(len(X) * self.val_size)
            
            X_train = X[:train_size]
            y_train = y[:train_size]
            X_val = X[train_size:train_size + val_size]
            y_val = y[train_size:train_size + val_size]
            X_test = X[train_size + val_size:]
            y_test = y[train_size + val_size:]
            
            print(f"训练集大小: {len(X_train)}, 验证集大小: {len(X_val)}, 测试集大小: {len(X_test)}")
            
            # 创建数据集和数据加载器
            train_dataset = WeatherDataset(X_train, y_train)
            val_dataset = WeatherDataset(X_val, y_val)
            test_dataset = WeatherDataset(X_test, y_test)
            
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
            
            self.datasets[var_name] = {
                'train': train_dataset, 'val': val_dataset, 'test': test_dataset
            }
            self.dataloaders[var_name] = {
                'train': train_loader, 'val': val_loader, 'test': test_loader
            }
    
    def run_model_comparison(self, target_variable='Temperature'):
        """运行不同模型的对比实验"""
        print(f"\n开始 {target_variable} 的模型对比实验...")
        
        # 获取数据加载器
        train_loader = self.dataloaders[target_variable]['train']
        val_loader = self.dataloaders[target_variable]['val']
        test_loader = self.dataloaders[target_variable]['test']
        
        # 定义模型配置
        model_configs = {
            'Transformer': {
                'class': WeatherTransformer,
                'params': {
                    'input_dim': self.input_dim,
                    'd_model': 256,
                    'nhead': 8,
                    'num_layers': 6,
                    'seq_length': self.seq_length,
                    'pred_length': self.pred_length,
                    'target_dim': 1,
                    'dropout': 0.1
                }
            },
            'LocalAttentionTransformer': {
                'class': LocalAttentionTransformer,
                'params': {
                    'input_dim': self.input_dim,
                    'd_model': 256,
                    'nhead': 8,
                    'num_layers': 6,
                    'seq_length': self.seq_length,
                    'pred_length': self.pred_length,
                    'target_dim': 1,
                    'dropout': 0.1,
                    'local_window': 48
                }
            },
            'CNNTransformer': {
                'class': CNNTransformer,
                'params': {
                    'input_dim': self.input_dim,
                    'd_model': 256,
                    'nhead': 8,
                    'num_layers': 4,
                    'seq_length': self.seq_length,
                    'pred_length': self.pred_length,
                    'target_dim': 1,
                    'dropout': 0.1
                }
            },
            'LSTM': {
                'class': WeatherLSTM,
                'params': {
                    'input_dim': self.input_dim,
                    'hidden_dim': 256,
                    'num_layers': 3,
                    'pred_length': self.pred_length,
                    'target_dim': 1,
                    'dropout': 0.1
                }
            }
        }
        
        # 存储结果
        model_results = {}
        
        # 训练和评估每个模型
        for model_name, config in model_configs.items():
            print(f"\n训练 {model_name} 模型...")
            
            try:
                # 创建模型
                model = config['class'](**config['params'])
                
                # 训练模型 - 使用更低的学习率
                trainer = ModelTrainer(model, self.device, learning_rate=0.00005)
                trained_model = trainer.train(train_loader, val_loader, epochs=30, patience=8)
                
                # 评估模型
                evaluator = ModelEvaluator(trained_model, self.device)
                predictions, actuals = evaluator.predict(test_loader)
                metrics = evaluator.evaluate_metrics(predictions, actuals)
                
                # 存储结果
                model_results[model_name] = {
                    'model': trained_model,
                    'predictions': predictions,
                    'actuals': actuals,
                    'metrics': metrics,
                    'train_losses': trainer.train_losses,
                    'val_losses': trainer.val_losses
                }
                
                print(f"{model_name} 整体性能:")
                print(f"  RMSE: {metrics['overall']['RMSE']:.4f}")
                print(f"  MAE: {metrics['overall']['MAE']:.4f}")
                print(f"  R²: {metrics['overall']['R2']:.4f}")
                
            except Exception as e:
                print(f"{model_name} 训练失败: {e}")
                continue
        
        return model_results
    
    def hyperparameter_tuning(self, target_variable='Temperature', model_class=WeatherTransformer):
        """进行超参数调优"""
        print(f"\n开始 {target_variable} 的超参数调优...")
        
        # 获取数据加载器
        train_loader = self.dataloaders[target_variable]['train']
        val_loader = self.dataloaders[target_variable]['val']
        
        # 定义超参数搜索空间 - 减少搜索空间以加快调优速度
        param_grid = {
            'd_model': [128, 256],  # 减少选项
            'nhead': [4, 8],        # 减少选项
            'num_layers': [3, 6],   # 减少选项
            'learning_rate': [0.0001, 0.001],  # 降低学习率选项
            'dropout': [0.1, 0.2]   # 减少选项
        }
        
        # 进行调优
        tuner = HyperparameterTuner(model_class, train_loader, val_loader, self.device)
        best_params, best_score, all_results = tuner.tune_hyperparameters(
            param_grid, self.input_dim, target_dim=1
        )
        
        print(f"\n最佳参数: {best_params}")
        print(f"最佳验证损失: {best_score:.6f}")
        
        return best_params, best_score, all_results
    
    def analyze_variable_performance(self, model_results):
        """分析不同气象变量的预测性能"""
        print("\n分析不同气象变量的预测性能...")
        
        variable_performance = {}
        
        for var_name in self.target_variables.keys():
            print(f"\n分析 {var_name}...")
            
            # 对每个变量运行模型对比
            results = self.run_model_comparison(var_name)
            variable_performance[var_name] = results
            
            # 输出性能总结
            print(f"\n{var_name} 性能总结:")
            for model_name, result in results.items():
                metrics = result['metrics']['overall']
                print(f"  {model_name}: RMSE={metrics['RMSE']:.4f}, R²={metrics['R2']:.4f}")
        
        return variable_performance
    
    def plot_comparison_results(self, model_results, target_variable='Temperature'):
        """绘制模型对比结果"""
        print(f"\n绘制 {target_variable} 的对比结果...")
        
        # 性能对比图
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 提取指标
        models = list(model_results.keys())
        rmse_scores = [model_results[m]['metrics']['overall']['RMSE'] for m in models]
        mae_scores = [model_results[m]['metrics']['overall']['MAE'] for m in models]
        r2_scores = [model_results[m]['metrics']['overall']['R2'] for m in models]
        
        # RMSE对比
        axes[0, 0].bar(models, rmse_scores)
        axes[0, 0].set_title('RMSE Comparison')
        axes[0, 0].set_ylabel('RMSE')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # MAE对比
        axes[0, 1].bar(models, mae_scores)
        axes[0, 1].set_title('MAE Comparison')
        axes[0, 1].set_ylabel('MAE')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # R²对比
        axes[1, 0].bar(models, r2_scores)
        axes[1, 0].set_title('R² Score Comparison')
        axes[1, 0].set_ylabel('R² Score')
        axes[1, 0].tick_params(axis='x', rotation=45)
        
        # 训练损失曲线
        for model_name, result in model_results.items():
            if 'train_losses' in result:
                axes[1, 1].plot(result['train_losses'], label=f'{model_name} Train')
                axes[1, 1].plot(result['val_losses'], label=f'{model_name} Val', linestyle='--')
        
        axes[1, 1].set_title('Training Loss Curves')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.savefig(f'{target_variable}_model_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # 绘制预测结果
        for model_name, result in model_results.items():
            evaluator = ModelEvaluator(result['model'], self.device)
            evaluator.plot_predictions(
                result['predictions'], 
                result['actuals'], 
                [target_variable],
                save_path=f'{target_variable}_{model_name}_predictions.png'
            )
    
    def generate_report(self, all_results):
        """生成实验报告"""
        print("\n生成实验报告...")
        
        report = f"""
# 气象数据时序预测实验报告

## 实验配置
- 输入序列长度: {self.seq_length} 小时
- 预测长度: {self.pred_length} 小时
- 数据特征维度: {self.input_dim}
- 设备: {self.device}

## 模型性能总结

"""
        
        for var_name, results in all_results.items():
            report += f"\n### {var_name} 预测性能\n\n"
            report += "| 模型 | RMSE | MAE | R² |\n"
            report += "|------|------|-----|----|\n"
            
            for model_name, result in results.items():
                metrics = result['metrics']['overall']
                report += f"| {model_name} | {metrics['RMSE']:.4f} | {metrics['MAE']:.4f} | {metrics['R2']:.4f} |\n"
        
        report += """

## 实验结论

1. **模型性能对比**: Transformer系列模型在长序列预测上表现更好
2. **局部注意力改进**: 局部注意力机制能够提高计算效率
3. **CNN-Transformer协同**: 结合CNN的特征提取能力提升预测精度
4. **变量适用性**: 不同气象变量对模型的敏感性不同

## 建议

1. 针对不同气象变量选择合适的模型架构
2. 根据预测时长调整序列长度和模型复杂度
3. 考虑引入更多气象物理约束
"""
        
        # 保存报告
        with open('weather_forecasting_report.md', 'w', encoding='utf-8') as f:
            f.write(report)
        
        print("报告已保存到 weather_forecasting_report.md")
        return report

def main():
    """主函数"""
    print("启动气象数据时序预测实验...")
    
    # 初始化实验
    experiment = WeatherForecastingExperiment(
        data_path='weather.csv',  # 请替换为实际数据路径
        seq_length=168,  # 7天
        pred_length=24   # 24小时
    )
    
    try:
        # 1. 加载和准备数据
        experiment.load_and_prepare_data()
        
        # 2. 超参数调优（可选）
        print("\n是否进行超参数调优？(y/n)")
        user_input = input().lower()
        if user_input == 'y':
            # 使用快速调优 - 调用独立函数
            best_params, best_score, tuning_results = quick_hyperparameter_tuning(experiment)
            print(f"调优完成，最佳参数: {best_params}")
        
        # 3. 运行模型对比实验
        print("\n运行模型对比实验...")
        temperature_results = experiment.run_model_comparison('Temperature')
        
        # 4. 绘制对比结果
        experiment.plot_comparison_results(temperature_results, 'Temperature')
        
        # 5. 分析所有气象变量的性能
        print("\n是否分析所有气象变量？(y/n)")
        user_input = input().lower()
        if user_input == 'y':
            all_variable_results = experiment.analyze_variable_performance(temperature_results)
        else:
            all_variable_results = {'Temperature': temperature_results}
        
        # 6. 生成实验报告
        report = experiment.generate_report(all_variable_results)
        print("\n实验完成！")
        print(report)
        
    except Exception as e:
        print(f"实验过程中发生错误: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

启动气象数据时序预测实验...
使用设备: cuda
正在加载和预处理数据...
数据集列名: ['Formatted Date', 'Summary', 'Precip Type', 'Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)', 'Visibility (km)', 'Loud Cover', 'Pressure (millibars)', 'Daily Summary']
前5行数据:
                  Formatted Date        Summary Precip Type  Temperature (C)  \
0  2006-04-01 00:00:00.000 +0200  Partly Cloudy        rain         9.472222   
1  2006-04-01 01:00:00.000 +0200  Partly Cloudy        rain         9.355556   
2  2006-04-01 02:00:00.000 +0200  Mostly Cloudy        rain         9.377778   
3  2006-04-01 03:00:00.000 +0200  Partly Cloudy        rain         8.288889   
4  2006-04-01 04:00:00.000 +0200  Mostly Cloudy        rain         8.755556   

   Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
0                  7.388889      0.89            14.1197   
1                  7.227778      0.86            14.2646   
2                  9.377778      0.89             3.9284   

KeyboardInterrupt: 